In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
from math import e
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from keras.models import load_model
from sklearn.decomposition import PCA

In [2]:
# AO Calculations

modelName=['IPSL-CM6A-LR','MPI-ESM1-2-LR','EC-Earth3','ACCESS-CM2','IITM-ESM']

for m in modelName :

    # Calculate AO - input std normal latitude wighted 1000mb z
    ZData = xr.open_dataset('../../../Data/ssp370/'+m+'/processed/weighted1000Std.nc')
    #SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

    ZData.zg.values
    ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
    ZData.zg.values

    t1=ZData.zg.stack(z=("lat", "lon"))
    # fit scaler on training data
    zg_PC =PCA().fit_transform(t1)

    print(zg_PC[:,0].shape)
    AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


    norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
    aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
    aoX.shape

    AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})

    AO_ds.to_netcdf(m+'-AOindex-NDJF-Daily-2015-2100.nc')
    AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

    AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

(10341,)
(10341,)
(10341,)
(10341,)
(10101,)
